# 爬虫

## 简介

爬取[萌娘百科](https://mzh.moegirl.org.cn/)。从GalGame公司页面和动画公司页面开始。

由于使用`action=raw`参数可以获得页面源内容，所以使用`requests`获取页面源内容，同时获取状态码，不是200则不爬取。

由于该页面有动态加载部分，为了实现链接分析，获取页面上所有内容，所以使用`selenium`爬取页面内容。

In [1]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from urllib.parse import quote,unquote
import requests
import re
import time
import os
import csv
import json
import pickle

In [ ]:
maxSpiderTime = 100000
service = Service("G:\Python\edgedriver_win64\msedgedriver.exe")
options = webdriver.EdgeOptions()
options.add_argument("headless")
driver = webdriver.Edge(service=service,options=options)
driver.set_page_load_timeout(3)
driver.set_script_timeout(3)

In [2]:
request_headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.58",
    "Connection": "keep-alive",
}
# url = "https://mzh.moegirl.org.cn/index.php?title=Special%3ARandom&utm_source=moe_homeland"
url = ["https://mzh.moegirl.org.cn/index.php?utm_medium=simple_list&title=Template%3AGalgame%E5%85%AC%E5%8F%B8&utm_source=moe_homeland","https://mzh.moegirl.org.cn/index.php?utm_medium=simple_list&title=Template%3A%E5%8A%A8%E7%94%BB%E5%85%AC%E5%8F%B8&utm_source=moe_homeland"]
url_info = {}
urls_not_found = set()
urls_company = set()
urls_falid = set()
urls_pages = set()
sleepTime = 0

In [3]:
# 第一次爬取：爬取GalGame、ACG和动画公司信息
def first_spider():
    for i in url:
        try:
            getPage = False
            while not getPage:
                try:
                    driver.get(url=i)
                except:
                    time.sleep(sleepTime)
                link_elements = driver.find_elements(By.TAG_NAME,"a")
                if len(link_elements) != 0:
                    getPage = True
            for link in link_elements:
                link_href = link.get_attribute("href")
                if link_href == None or link_href == '':
                    continue
                one_url = unquote(link_href)
                if one_url[0] == '/':
                    one_url = "https://mzh.moegirl.org.cn" + one_url
                try:
                    resp = requests.get(url=one_url,headers=request_headers)
                except:
                    continue
                if not resp.status_code == 200:
                    urls_not_found.add(one_url)
                    continue
                urls_company.add(one_url)
                if one_url in url_info.keys():
                    if link.text.strip() != "":
                        url_info[one_url]["anchor_text"].add(link.text.strip())
                else:
                    url_info[one_url] = {}
                    url_info[one_url]["url"] = one_url
                    url_info[one_url]["anchor_text"] = set()
                    url_info[one_url]["title"] = ""
                    url_info[one_url]["page"] = 0
                    url_info[one_url]["page_rank"] = 0
        except:
            continue

In [4]:
# 爬取网页内容
def second_spider(urls, add_link=True, store_link=False):
    sleepTime = 0
    for i in urls:
        try:
            try:
                driver.get(url=i)
            except:
                time.sleep(sleepTime)
                if i not in url_info.keys():
                    url_info[i] = {}
                    url_info[i]["url"] = i
                    url_info[i]["anchor_text"] = set()
                    url_info[i]["title"] = ""
                    url_info[i]["page"] = 0
                    url_info[i]["page_rank"] = 0
                # 获取标题信息
                url_info[i]["title"] = driver.find_element(By.TAG_NAME,"h1").text
                # 获取页面内容并写入文件
                try:
                    content = driver.find_element(By.CSS_SELECTOR,'#mw-body')
                    sleepTime = 0
                except:
                    urls_falid.add(i)
                    sleepTime += 1
                    continue
                timeNow = int(time.mktime(time.localtime(time.time())))
                with open('spider_pages/' + str(timeNow), 'w', encoding='utf-8') as f:
                    f.write(content.text)
                url_info[i]["page"] = timeNow
                if store_link:
                    with open("spider_pages/" + str(timeNow) + "_to_link", 'w', encoding='utf-8') as f:
                        for link in driver.find_elements(By.TAG_NAME,"a"):
                            try:
                                link_href = link.get_attribute("href")
                            except:
                                continue
                            if link_href == None or link_href == '':
                                continue
                            one_url = unquote(link_href)
                            f.write(one_url)
                            f.write('\n')
                if not add_link:
                    continue
                for link in driver.find_elements(By.TAG_NAME,"a"):
                    link_href = link.get_attribute("href")
                    if link_href == None or link_href == '':
                        continue
                    one_url = unquote(link_href)
                    if one_url[0] == '/':
                        one_url = "https://mzh.moegirl.org.cn" + one_url
                    # 如果是无法获取或者公司的链接，则continue
                    if one_url in urls_not_found or one_url in urls_company:
                        continue
                    urls_pages.add(one_url)
                    if one_url in url_info.keys():
                        url_info[one_url]["anchor_text"].add(link.text.strip())
                    else:
                        url_info[one_url] = {}
                        url_info[one_url]["url"] = one_url
                        url_info[one_url]["anchor_text"] = set()
                        url_info[one_url]["title"] = ""
                        url_info[one_url]["page"] = 0
                        url_info[one_url]["page_rank"] = 0
        except:
            continue

In [5]:
import re
# 加上action=raw，爬取源文件
def second_spider_raw(urls, add_link=True, store_link=False, over_write = False):
    sleepTime = 0
    for i in urls:
        try:
            split_url = re.split('[#?&]', i)
            resp = requests.get(url=split_url[0] + "?action=raw",headers=request_headers)
            content = resp.text
            sleepTime = 0
        except:
            urls_falid.add(i)
            sleepTime += 1
            continue
        if resp.status_code != 200:
            continue
        if not over_write:
            if i in url_info.keys() and url_info[i]["page"] != 0:
                continue
        try:
            try:
                driver.get(url=i)
            except:
                time.sleep(sleepTime)
                if i not in url_info.keys():
                    url_info[i] = {}
                    url_info[i]["url"] = i
                    url_info[i]["anchor_text"] = set()
                    url_info[i]["title"] = ""
                    url_info[i]["page"] = 0
                    url_info[i]["page_rank"] = 0
                # 获取标题信息
                url_info[i]["title"] = driver.find_element(By.TAG_NAME,"h1").text
                timeNow = int(time.mktime(time.localtime(time.time())))
                with open('spider_pages/' + str(timeNow), 'w', encoding='utf-8') as f:
                    f.write(content)
                url_info[i]["page"] = timeNow
                if store_link:
                    with open("spider_pages/" + str(timeNow) + "_to_link", 'w', encoding='utf-8') as f:
                        for link in driver.find_elements(By.TAG_NAME,"a"):
                            try:
                                link_href = link.get_attribute("href")
                            except:
                                continue
                            if link_href == None or link_href == '':
                                continue
                            one_url = unquote(link_href)
                            res = re.split('[#?&]', one_url)
                            f.write(res[0])
                            f.write('\n')
                if not add_link:
                    continue
                for link in driver.find_elements(By.TAG_NAME,"a"):
                    link_href = link.get_attribute("href")
                    if link_href == None or link_href == '':
                        continue
                    one_url = unquote(link_href)
                    if one_url[0] == '/':
                        one_url = "https://mzh.moegirl.org.cn" + one_url
                    res = re.split('[#?&]', one_url)
                    one_url = res[0]
                    # 如果是无法获取或者公司的链接，则continue
                    if one_url in urls_not_found or one_url in urls_company:
                        continue
                    urls_pages.add(one_url)
                    if one_url in url_info.keys():
                        url_info[one_url]["anchor_text"].add(link.text.strip())
                    else:
                        url_info[one_url] = {}
                        url_info[one_url]["url"] = one_url
                        url_info[one_url]["anchor_text"] = set()
                        url_info[one_url]["title"] = ""
                        url_info[one_url]["page"] = 0
                        url_info[one_url]["page_rank"] = 0
        except:
            continue

## 第一次爬虫

从两个页面爬取GalGame公司和动画公司信息。

In [6]:
first_spider()

### 读取文件

由于在爬虫时需要保存数据，所以有的时候会需要从保存的数据中读取信息，以下为读取信息的代码。

In [3]:
with open("page_info.json", "r", encoding='utf-8') as f:
    url_info = json.load(f)
    for key,value in url_info.items():
        url_info[key]["anchor_text"] = set(value["anchor_text"])

# with open("urls_company.pk","rb") as f:
#     urls_company = pickle.load(f)

# with open("urls_not_found.pk", "rb") as f:
#     urls_not_found = pickle.load(f)

# with open("urls_pages.pk", "rb") as f:
#     urls_pages = pickle.load(f)

### 写入文件

第一次爬取信息时需要写入文件，备份。

In [7]:
# with open("page_info.json", "w", encoding='utf-8') as f:
#     for key,value in url_info.items():
#         url_info[key]["anchor_text"] = list(value["anchor_text"])
#     json.dump(url_info, f, indent=4, ensure_ascii=False)
#     for key,value in url_info.items():
#         url_info[key]["anchor_text"] = set(value["anchor_text"])

# with open("urls_company.pk","wb") as f:
#     pickle.dump(urls_company, f)

# with open("urls_not_found.pk", "wb") as f:
#     pickle.dump(urls_not_found, f)

### 删除其他站点的数据

每轮爬取结束后都需要删除其他站点数据。

In [ ]:
def del_other_link():
    keys = list(url_info.keys())
    for i in keys:
        if re.match(r'https://mzh.moegirl.org.cn/.*', i):
            continue
        else:
            p = url_info.pop(i)

In [14]:
del_other_link()
len(url_info)

18367

In [15]:
# with open("page_info.json", "w", encoding='utf-8') as f:
#     for key,value in url_info.items():
#         url_info[key]["anchor_text"] = list(value["anchor_text"])
#     json.dump(url_info, f, indent=4, ensure_ascii=False)
#     for key,value in url_info.items():
#         url_info[key]["anchor_text"] = set(value["anchor_text"])

## 第二次爬取

第一次爬取之后，获得了465个公司的信息，第二次需要在每个公司页面爬取链接。

In [8]:
second_spider(urls_company, store_link=True)

In [9]:
second_spider(urls_falid, store_link=True)

In [10]:
# with open("urls_pages.pk", "wb") as f:
#     pickle.dump(urls_pages, f)

In [47]:
# import random

# testpage = random.choice(list(urls_pages))
# second_spider([testpage], False, True)

In [10]:
# url_info_new = {}
# for key,value in url_info.items():
#     res = re.split('[#?&]', key)
#     if len(res) == 0:
#         url_info_new[key] = value
#     else:
#         match = re.match(r'https://mzh.moegirl.org.cn/index.php\?title=.*', key)
#         if match:
#             continue
#         else:
#             url_info_new[res[0]] = value
#             url_info_new[res[0]]["url"] = res[0]

# len(url_info_new)

20310

备份数据

In [ ]:
# with open("page_info.json", "w", encoding='utf-8') as f:
#     for key,value in url_info.items():
#         url_info[key]["anchor_text"] = list(value["anchor_text"])
#     json.dump(url_info, f, indent=4, ensure_ascii=False)
#     for key,value in url_info.items():
#         url_info[key]["anchor_text"] = set(value["anchor_text"])

## 第三次爬取

对于第二次获取到的所有公司的作品，进行第三次爬取，保存页面内容和页面中的链接，但是不增加爬取链接。

### 爬取之前处理

由于爬取的四万余链接中，大部分为无效链接，所以对链接进行预处理。

设定过滤链接，移除所有与过滤链接匹配的链接，以正则表达式给出。

In [13]:
url_filter = [
    r"https://mzh.moegirl.org.cn/Special:.*", 
    r"https://mzh.moegirl.org.cn/index.php?title=.*", 
    r".*\.jpg", 
    r".*\.png", 
    r"https://mzh.moegirl.org.cn/萌娘百科:.*",
    r"https://mzh.moegirl.org.cn/萌娘百科_talk:"
    ]

keys = list(url_info.keys())

for key in keys:
    for i in url_filter:
        if re.match(i, key):
            m = url_info.pop(key)

len(url_info)

15963

In [11]:
# with open("page_info_new.json", "w", encoding='utf-8') as f:
#     for key,value in url_info_new.items():
#         url_info_new[key]["anchor_text"] = list(value["anchor_text"])
#     json.dump(url_info_new, f, indent=4, ensure_ascii=False)
#     for key,value in url_info_new.items():
#         url_info_new[key]["anchor_text"] = set(value["anchor_text"])

In [4]:
urls_pages = list(url_info.keys())

由于多次中断了爬虫，所以在总的链接中会有已爬取的和未爬取的链接，需要将未爬取的链接取出。

In [5]:
url_not_search = []
for i in urls_pages:
    if url_info[i]["page"] == 0:
        url_not_search.append(i)

len(url_not_search)

0

In [13]:
with open("url_not_search.json", "w", encoding="utf-8") as f:
    json.dump(url_not_search, f, indent=4, ensure_ascii=False)

In [9]:
url_not_search[0:10]

['https://mzh.moegirl.org.cn/Template:Galgame公司',
 'https://mzh.moegirl.org.cn/COOL',
 'https://mzh.moegirl.org.cn/松竹',
 'https://mzh.moegirl.org.cn/糖果盒动画',
 'https://mzh.moegirl.org.cn/童梦',
 'https://mzh.moegirl.org.cn/晓(公司)',
 'https://mzh.moegirl.org.cn/亚细亚堂',
 'https://mzh.moegirl.org.cn/月虹(公司)',
 'https://mzh.moegirl.org.cn/朱夏',
 'https://mzh.moegirl.org.cn/ALCOO']

按1000个链接分组，每跑完一组就保存一次链接和页面的信息（`page_info`）。

In [10]:
total = len(url_not_search)
for i in range(0, total, 1000):
    next = i + 1000
    if next > total:
        next = total
    second_spider_raw(url_not_search[i:next], False, True)
    with open("page_info{}.json".format(i), "w", encoding='utf-8') as f:
        for key,value in url_info.items():
            url_info[key]["anchor_text"] = list(value["anchor_text"])
            if "" in url_info[key]["anchor_text"]:
                url_info[key]["anchor_text"].remove("")
        json.dump(url_info, f, indent=4, ensure_ascii=False)
        for key,value in url_info.items():
            url_info[key]["anchor_text"] = set(value["anchor_text"])

### 爬取完成后处理

爬取完成后，将所有无法爬取的链接从字典中移除

In [15]:
keys = list(url_info.keys())
for key in keys:
    if url_info[key]["page"] == 0:
        m = url_info.pop(key)

len(url_info)

17484

In [15]:
with open("page_info.json", "w", encoding='utf-8') as f:
    for key,value in url_info.items():
        url_info[key]["anchor_text"] = list(value["anchor_text"])
    json.dump(url_info, f, indent=4, ensure_ascii=False)
    for key,value in url_info.items():
        url_info[key]["anchor_text"] = set(value["anchor_text"])

In [ ]:
with open("urls_pages.pk", "wb") as f:
    pickle.dump(urls_pages, f)

### 处理页面

在爬取页面过程中，可能会出现页面不被需要的情况，所以将所有有用的页面复制到另一个文件夹中。

页面处理分为两方面：

+ 将所有属于最后链接对应的页面移动到另一个文件夹中。
+ 读取页面链接，将处理我们爬取的链接的链接写入另一个文件夹中的对应文件中。

In [19]:
import shutil

for key, value in url_info.items():
    page_name = str(value["page"])
    m = shutil.copy("spider_pages/" + page_name, "new_pages/" + page_name)
    
    try:
        with open("spider_pages/" + page_name + "_to_link", "r", encoding='utf-8') as f:
            to_urls = f.readlines()
    except:
        with open("page_link_num/" + page_name, "w", encoding='utf-8') as f:
            m = json.dump([], f, indent=4, ensure_ascii=False)
        continue

    to_urls_new = set()
    for i in to_urls:
        res = re.split('[#?&]', i)
        add_url = res[0].strip()
        if add_url in url_info.keys() and add_url != "https://mzh.moegirl.org.cn/index.php":
            to_urls_new.add(url_info[add_url]["page"])
    
    with open("page_link_num/" + page_name, "w", encoding='utf-8') as f:
        m = json.dump(list(to_urls_new), f, indent=4, ensure_ascii=False)

### 链接分析

为了后续链接分析的方便，我们在这里构建链接关系图。

从链接关系构建链接图，保存图中的所有边，为`link_graph.pk`。保存格式为
```
{
    (int)from1:[(int)to1, (int)to2,...],
    (int)from2:[(int)to3, (int)to4,...],
    ...
}
```

In [22]:
link_edges = os.listdir("page_link_num")
len(link_edges)
link_graph = {}
for i in link_edges:
    with open("page_link_num/" + i, "r", encoding="utf-8") as f:
        edges = json.load(f)
    link_graph[int(i)] = edges

15963

In [23]:
with open("link_graph.pk", "wb") as f:
    pickle.dump(link_graph, f)

为了方便，我们设计`num2link.pk`文件，构建从页面到链接的映射关系：

In [24]:
num2link = {}
for key, value in url_info.items():
    num2link[value["page"]] = key

with open("num2link.pk", "wb") as f:
    pickle.dump(num2link, f)